In [ ]:
import os, json
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4,5"

In [ ]:
import os, random
import numpy as np
import pandas as pd
from datetime import datetime
import torch


ROUNDS = 10
ATTACKS_PER_ROUND = 20
SAMPLE_CAP = 1000
OUTPUT_DIR = "./content/experiment_jailbreak"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# toggles
USE_PARAPHRASER = True 

# dataset
DATASET_ID = "JailbreakV-28K/JailBreakV-28k"

# random seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
from datasets import load_dataset

print("Loading dataset...")

if DATASET_ID == "JailbreakBench/JBB-Behaviors":
    ds_full = load_dataset(DATASET_ID, "behaviors")["harmful"]
    PROMPT_FIELD = "Goal"
else:
    ds_full = load_dataset("JailbreakV-28K/JailBreakV-28k", "JailBreakV_28K")["JailBreakV_28K"]
    PROMPT_FIELD = "jailbreak_query"

print("Dataset loaded. Number of examples:", len(ds_full))

first = ds_full[0]
print("Available fields:", list(first.keys()))
print("Using prompt field:", PROMPT_FIELD)

In [ ]:
a_prompts = []
seen = set()

count = 0
for item in ds_full:
  if count >= SAMPLE_CAP:
        break
  raw = item.get(PROMPT_FIELD, None)
  if raw and raw not in seen:
    a_prompts.append(raw)
    seen.add(raw)
  count += 1
print("Collected prompts:", len(a_prompts))

In [ ]:
USE_REAL_PARAPH = USE_PARAPHRASER
hf_paraphraser = None
PARAPH_MODEL = "Vamsi/T5_Paraphrase_Paws"

if USE_REAL_PARAPH:
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    try:
        print("Loading paraphraser...")
        par_tok = AutoTokenizer.from_pretrained(PARAPH_MODEL)

        par_model = AutoModelForSeq2SeqLM.from_pretrained(
            PARAPH_MODEL,
            torch_dtype=torch.float16
        )
        if torch.cuda.is_available():
            par_model.to("cuda")

        par_model.eval()
        hf_paraphraser = (par_tok, par_model)
        print("Paraphraser loaded.")
    except Exception as e:
        print("Could not load paraphraser, using fallback:", e)
        USE_REAL_PARAPH = False


def paraphrase_text(prompt, n=4):
    if USE_REAL_PARAPH and hf_paraphraser:
        tok, model = hf_paraphraser

        inputs = tok(
            "paraphrase: " + prompt + " </s>",
            return_tensors="pt",
            truncation=True
        )
        # put inputs on SAME device as the model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # generate paraphrases
        gen = model.generate(
            **inputs,
            do_sample=True,
            num_return_sequences=n,
            top_p=0.95,
            max_new_tokens=1000
        )

        out = [tok.decode(g, skip_special_tokens=True).strip() for g in gen]

        seen = []
        for o in out:
            if o not in seen:
                seen.append(o)

        return seen[:n]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
OFFLOAD_DIR = "./content/mpt_offload" 
os.makedirs(OFFLOAD_DIR, exist_ok=True)

def load_llms():
    MISTRAL_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

    mistral_tokenizer = AutoTokenizer.from_pretrained(
        MISTRAL_MODEL_NAME,
        trust_remote_code=True
    )

    mistral_model = AutoModelForCausalLM.from_pretrained(
        MISTRAL_MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
        offload_folder=OFFLOAD_DIR,
        trust_remote_code=True
    )

    mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
    mistral_model.config.pad_token_id = mistral_model.config.eos_token_id
    mistral_model.eval()

    LLAMA_MODEL_ID = "meta-llama/Llama-3.1-8B"

    llama_pipeline = pipeline(
        "text-generation",
        model=LLAMA_MODEL_ID,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )

    return {
        "mistral": {
            "tokenizer": mistral_tokenizer,
            "model": mistral_model,
        },
        "llama": llama_pipeline,
    }

models = load_llms()

mistral_tok = models["mistral"]["tokenizer"]
mistral_model = models["mistral"]["model"]
llama = models["llama"]

In [ ]:
def query_mistral_llm(prompt, max_new_tokens=1000):
    """Generates a model reply from the LLM"""
    inputs = mistral_tok(prompt, return_tensors="pt")
    with torch.no_grad():
        gen = mistral_model.generate(**inputs, max_new_tokens=max_new_tokens)
    output = mistral_tok.decode(gen[0], skip_special_tokens=True)
    return output.replace(prompt, "", 1).strip()

In [ ]:
def query_llama_llm(prompt_text: str) -> str:
    out = llama(
        prompt_text,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
    )
    return out[0]["generated_text"]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM


SAFETY_MODEL_NAME = "meta-llama/LlamaGuard-7B"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_safe = AutoTokenizer.from_pretrained(SAFETY_MODEL_NAME)
model_safe = AutoModelForCausalLM.from_pretrained(
    SAFETY_MODEL_NAME,
    torch_dtype=torch.float16,
    device_map='auto'
)

model_safe.eval()


def classify_llamaguard(text, max_new_tokens=1000):
    """
    Run LlamaGuard on a single text string and return its raw output text.
    """
    chat = [
        {"role": "user", "content": text}
    ]

    input_ids = tokenizer_safe.apply_chat_template(
        chat,
        return_tensors="pt"
    )

    with torch.no_grad():
        output_ids = model_safe.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer_safe.eos_token_id,
            do_sample=False
        )

    prompt_len = input_ids.shape[-1]
    generated_ids = output_ids[0][prompt_len:]
    generated_text = tokenizer_safe.decode(generated_ids, skip_special_tokens=True)

    return generated_text


def is_unsafe_reply_llamaguard(text):

    raw = classify_llamaguard(text)
    return raw != 'safe'


In [ ]:
os.makedirs("results", exist_ok=True)
os.makedirs("plots", exist_ok=True)

mistral_history = []
mistral_global_attacks = []

llama_history = []
llama_global_attacks = []

SEED_PROMPTS = 5
SAVE_ONLY_SUCCESS = False 

current_bases = random.sample(a_prompts, k=min(SEED_PROMPTS, len(a_prompts)))

print(f"Starting iterative experiment:")
print(f"ROUNDS={ROUNDS}, SEED_PROMPTS={len(current_bases)}, ATTACKS_PER_ROUND={ATTACKS_PER_ROUND}")


for r in range(1, ROUNDS + 1):
    print(f"\n=== Round {r} ===")
    attacks = []

    # Generate paraphrased variants from the current base prompts
    for b in current_bases:
        variants = paraphrase_text(b, n=3)
        if variants:
            attacks.extend(variants)

    if ATTACKS_PER_ROUND is not None:
        attacks = attacks[:ATTACKS_PER_ROUND]

    if not attacks:
        print("No attacks generated; stopping.")
        break

    mistral_successes = []
    llama_successes = []

    for prompt_text in attacks:

        try:
            model_reply = query_mistral_llm(prompt_text)
            flagged = is_unsafe_reply_llamaguard(model_reply)
        except Exception as e:
            print("Error querying Mistral:", e)
            model_reply = None
            flagged = False

        record = {
            "round": r,
            "prompt": prompt_text,
            "model_reply": model_reply,
            "unsafe_marked": flagged,
            "timestamp": datetime.utcnow().isoformat(),
        }

        if SAVE_ONLY_SUCCESS:
            if flagged:
                mistral_global_attacks.append(record)
        else:
            mistral_global_attacks.append(record)

        if flagged:
            mistral_successes.append(prompt_text)

        try:
            llama_reply = query_llama_llm(prompt_text)
            llama_flagged = is_unsafe_reply_llamaguard(llama_reply)
        except Exception as e:
            print("Error querying LLaMA:", e)
            llama_reply = None
            llama_flagged = False

        llama_record = {
            "round": r,
            "prompt": prompt_text,
            "model_reply": llama_reply,
            "unsafe_marked": llama_flagged,
            "timestamp": datetime.utcnow().isoformat(),
        }

        if SAVE_ONLY_SUCCESS:
            if llama_flagged:
                llama_global_attacks.append(llama_record)
        else:
            llama_global_attacks.append(llama_record)

        if llama_flagged:
            llama_successes.append(prompt_text)

    n_attacks = len(attacks)

    mistral_asr = len(mistral_successes) / max(1, n_attacks)
    llama_asr = len(llama_successes) / max(1, n_attacks)

    print(f"[Round {r}] Mistral: Attacks={n_attacks} Successes={len(mistral_successes)} ASR={mistral_asr:.4f}")
    print(f"[Round {r}] LLaMA:   Attacks={n_attacks} Successes={len(llama_successes)} ASR={llama_asr:.4f}")

    mistral_history.append({
        "round": r,
        "n_attacks": n_attacks,
        "n_success": len(mistral_successes),
        "ASR": mistral_asr,
    })
    llama_history.append({
        "round": r,
        "n_attacks": n_attacks,
        "n_success": len(llama_successes),
        "ASR": llama_asr,
    })

    with open("results/mistral_attacks.json", "w") as f:
        json.dump(mistral_global_attacks, f, indent=2)

    with open("results/llama_attacks.json", "w") as f:
        json.dump(llama_global_attacks, f, indent=2)

    with open("results/mistral_history.json", "w") as f:
        json.dump(mistral_history, f, indent=2)

    with open("results/llama_history.json", "w") as f:
        json.dump(llama_history, f, indent=2)

    current_bases = list(set(mistral_successes + llama_successes))

    if not current_bases:
        print("No successful attacks this round – stopping early.")
        break


In [ ]:
m_total = len(mistral_global_attacks)
m_success = sum(1 for r in mistral_global_attacks if r["unsafe_marked"])

l_total = len(llama_global_attacks)
l_success = sum(1 for r in llama_global_attacks if r["unsafe_marked"])

print("\n=== Overall summary ===")
print(f"Mistral: {m_success} / {m_total} successful attacks")
print(f"LLaMA:   {l_success} / {l_total} successful attacks")

m_asr_overall = m_success / max(1, m_total)
l_asr_overall = l_success / max(1, l_total)

m_asr_overall, l_asr_overall

summary = {
    "m_total": m_total,
    "m_success": m_success,
    "m_asr_overall": m_asr_overall,
    "l_total": l_total,
    "l_success": l_success,
    "l_asr_overall": l_asr_overall,
}

with open("results/overall_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

In [ ]:
import matplotlib.pyplot as plt
models = ["Mistral", "LLaMA"]
success_counts = [m_success, l_success]

plt.figure(figsize=(5, 4))
plt.bar(models, success_counts)
plt.title("Number of Successful Attacks")
plt.ylabel("# Successful Attacks")
plt.savefig("plots/success_counts.png", dpi=200, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
asr_values = [m_asr_overall, l_asr_overall]

plt.figure(figsize=(5, 4))
plt.bar(models, asr_values)
plt.title("Attack Success Rate (ASR)")
plt.ylabel("ASR")
plt.ylim(0, 1)
plt.savefig("plots/asr_overall.png", dpi=200, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
df_mistral = pd.DataFrame(mistral_history)
df_llama = pd.DataFrame(llama_history)

plt.figure(figsize=(6, 4))
plt.plot(df_mistral["round"], df_mistral["ASR"], marker="o")
plt.title("Mistral – ASR Over Rounds")
plt.xlabel("Round")
plt.ylabel("ASR")
plt.ylim(0, 1)
plt.grid(True)
plt.savefig("plots/mistral_asr_over_rounds.png", dpi=200, bbox_inches="tight") 
plt.show()
plt.close()


plt.figure(figsize=(6, 4))
plt.plot(df_llama["round"], df_llama["ASR"], marker="s")
plt.title("LLaMA – ASR Over Rounds")
plt.xlabel("Round")
plt.ylabel("ASR")
plt.ylim(0, 1)
plt.grid(True)
plt.savefig("plots/llama_asr_over_rounds.png", dpi=200, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(df_mistral["round"], df_mistral["ASR"], marker="o", label="Mistral")
plt.plot(df_llama["round"], df_llama["ASR"], marker="s", label="LLaMA")
plt.title("ASR Comparison: Mistral vs LLaMA Over Rounds")
plt.xlabel("Round")
plt.ylabel("ASR")
plt.ylim(0, 1)
plt.grid(True)
plt.legend()
plt.savefig("plots/asr_comparison_mistral_vs_llama.png", dpi=200, bbox_inches="tight")
plt.show()
plt.close()